# Part 1

In [116]:
!pip install -U -q PyDrive
!pip install cpt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import seaborn as sns
from cpt.cpt import Cpt

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [117]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [118]:
link = 'https://docs.google.com/spreadsheets/d/1RnGCGuKrOE3eXcmU9UXGooUYZW4aN85n/edit?usp=sharing&ouid=106829582408383186507&rtpof=true&sd=true'
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('bigbasketcasedataset.xlsx')  

In [119]:
df = pd.read_excel('bigbasketcasedataset.xlsx', 'Sheet1')
df.head()

,Member,Order,SKU,Date,Description
0,M09736,6468572,34993740,2014-09-22,Other Sauces
1,M09736,6468572,15669800,2014-09-22,Cashews
2,M09736,6468572,34989501,2014-09-22,Other Dals
3,M09736,6468572,7572303,2014-09-22,Namkeen
4,M09736,6468572,15669856,2014-09-22,Sugar


In [120]:
all_purchase = df[['Member','Date']].drop_duplicates().sort_values(['Member', 'Date'], ascending=[True, True])
# Frequency
count_purchase = df.groupby('Member').agg({"Date": "nunique"}).reset_index()
# Age on Platform
first_purchase = df.groupby('Member').Date.min().reset_index()
last_purchase = df.groupby('Member').Date.max().reset_index()


In [121]:
df2 = pd.merge(all_purchase, first_purchase, on='Member', how='left', suffixes=('_txn', '_first'))
df2 = pd.merge(df2, last_purchase, on='Member', how='left', suffixes=('_first', '_last'))
df2 = pd.merge(df2, count_purchase, on='Member', how='left', suffixes=('_last', '_freq'))

# Age on Platform
df2['datediff'] = (df2['Date_last'] - df2['Date_first']).dt.days

In [122]:
# This will be used to check the last 10 transactions to see recent txn patterns

df2['prev_txn'] = df2.groupby('Member')['Date_txn'].shift(1)
df2['prev_txn-1'] = df2.groupby('Member')['Date_txn'].shift(2)
df2['prev_txn-2'] = df2.groupby('Member')['Date_txn'].shift(3)
df2['prev_txn-3'] = df2.groupby('Member')['Date_txn'].shift(4)
df2['prev_txn-4'] = df2.groupby('Member')['Date_txn'].shift(5)
df2['prev_txn-5'] = df2.groupby('Member')['Date_txn'].shift(5)
df2['prev_txn-6'] = df2.groupby('Member')['Date_txn'].shift(6)
df2['prev_txn-7'] = df2.groupby('Member')['Date_txn'].shift(7)
df2['prev_txn-8'] = df2.groupby('Member')['Date_txn'].shift(8)
df2['prev_txn-9'] = df2.groupby('Member')['Date_txn'].shift(9)
df2['prev_txn-10'] = df2.groupby('Member')['Date_txn'].shift(10)

df2['diff_bw_txns'] = (df2['Date_txn'] - df2['prev_txn']).dt.days
df2['diff_bw_txns-1'] = (df2['Date_txn'] - df2['prev_txn-1']).dt.days
df2['diff_bw_txns-2'] = (df2['Date_txn'] - df2['prev_txn-2']).dt.days
df2['diff_bw_txns-3'] = (df2['Date_txn'] - df2['prev_txn-3']).dt.days
df2['diff_bw_txns-4'] = (df2['Date_txn'] - df2['prev_txn-4']).dt.days
df2['diff_bw_txns-5'] = (df2['Date_txn'] - df2['prev_txn-5']).dt.days
df2['diff_bw_txns-6'] = (df2['Date_txn'] - df2['prev_txn-6']).dt.days
df2['diff_bw_txns-7'] = (df2['Date_txn'] - df2['prev_txn-7']).dt.days
df2['diff_bw_txns-8'] = (df2['Date_txn'] - df2['prev_txn-8']).dt.days
df2['diff_bw_txns-9'] = (df2['Date_txn'] - df2['prev_txn-9']).dt.days
df2['diff_bw_txns-10'] = (df2['Date_txn'] - df2['prev_txn-10']).dt.days

df2.head()

,Member,Date_txn,Date_first,Date_last,Date_freq,datediff,prev_txn,prev_txn-1,prev_txn-2,prev_txn-3,prev_txn-4,prev_txn-5,prev_txn-6,prev_txn-7,prev_txn-8,prev_txn-9,prev_txn-10,diff_bw_txns,diff_bw_txns-1,diff_bw_txns-2,diff_bw_txns-3,diff_bw_txns-4,diff_bw_txns-5,diff_bw_txns-6,diff_bw_txns-7,diff_bw_txns-8,diff_bw_txns-9,diff_bw_txns-10
0,M04158,2012-04-12,2012-04-12,2014-12-03,128,965,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M04158,2012-10-12,2012-04-12,2014-12-03,128,965,2012-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M04158,2012-12-16,2012-04-12,2014-12-03,128,965,2012-10-12,2012-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,65.0,248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M04158,2012-12-26,2012-04-12,2014-12-03,128,965,2012-12-16,2012-10-12,2012-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,10.0,75.0,258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M04158,2013-01-04,2012-04-12,2014-12-03,128,965,2012-12-26,2012-12-16,2012-10-12,2012-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,9.0,19.0,84.0,267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
# # Mean and Variance in Purchases
# meanvar = df2.groupby('Member').agg({'diff_bw_txns': ['mean','std']}).reset_index()
# df2 = pd.merge(df2, meanvar, on='Member', how='left')
# df2.head()

In [124]:
df2 = df2.dropna()

In [125]:
import datetime as dt

df2['date'] = pd.to_datetime(df2['Date_txn'])
df2['date'] = df2['date'].map(dt.datetime.toordinal) 
df2['date'] = df2['date'] - df2['date'].min()
df3 = df2.drop(columns=['Date_first','Date_last','prev_txn','prev_txn-1','prev_txn-2','prev_txn-3','prev_txn-4','prev_txn-5','prev_txn-6','prev_txn-7','prev_txn-8','prev_txn-9','prev_txn-10','Date_txn'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [126]:
df3['Member'] = df3['Member'].str[1:]
df3['Member'] = df3['Member'].apply(pd.to_numeric)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7004 entries, 10 to 8063
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Member           7004 non-null   int64  
 1   Date_freq        7004 non-null   int64  
 2   datediff         7004 non-null   int64  
 3   diff_bw_txns     7004 non-null   float64
 4   diff_bw_txns-1   7004 non-null   float64
 5   diff_bw_txns-2   7004 non-null   float64
 6   diff_bw_txns-3   7004 non-null   float64
 7   diff_bw_txns-4   7004 non-null   float64
 8   diff_bw_txns-5   7004 non-null   float64
 9   diff_bw_txns-6   7004 non-null   float64
 10  diff_bw_txns-7   7004 non-null   float64
 11  diff_bw_txns-8   7004 non-null   float64
 12  diff_bw_txns-9   7004 non-null   float64
 13  diff_bw_txns-10  7004 non-null   float64
 14  date             7004 non-null   int64  
dtypes: float64(11), int64(4)
memory usage: 875.5 KB


In [127]:
df3.head()

,Member,Date_freq,datediff,diff_bw_txns,diff_bw_txns-1,diff_bw_txns-2,diff_bw_txns-3,diff_bw_txns-4,diff_bw_txns-5,diff_bw_txns-6,diff_bw_txns-7,diff_bw_txns-8,diff_bw_txns-9,diff_bw_txns-10,date
10,4158,128,965,3.0,8.0,11.0,13.0,14.0,14.0,17.0,26.0,36.0,101.0,284.0,323
11,4158,128,965,1.0,4.0,9.0,12.0,14.0,14.0,15.0,18.0,27.0,37.0,102.0,324
12,4158,128,965,4.0,5.0,8.0,13.0,16.0,16.0,18.0,19.0,22.0,31.0,41.0,328
13,4158,128,965,4.0,8.0,9.0,12.0,17.0,17.0,20.0,22.0,23.0,26.0,35.0,332
14,4158,128,965,2.0,6.0,10.0,11.0,14.0,14.0,19.0,22.0,24.0,25.0,28.0,334


In [128]:
Y = df3['diff_bw_txns']
X = df3.drop(columns=["diff_bw_txns"])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=29, stratify=X['Member'])

In [129]:
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB", GaussianNB()))
models.append(("RF", RandomForestRegressor()))
models.append(("SVC", SVC()))
models.append(("Dtree", DecisionTreeRegressor()))
models.append(("XGB", xgb.XGBRegressor(objective='reg:squarederror')))
models.append(("KNN",KNeighborsRegressor()))

In [130]:
for name, model in models:
    model.fit(X_train, Y_train)
    score = model.score(X_test, Y_test)
    # kfold = KFold(n_splits=2)
    # cv_result = cross_val_score(model, X_train, Y_train, cv = kfold, scoring = "accuracy")
    print(name, score)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LR 0.08993576017130621
NB 0.09707351891506068
RF 0.5603263226825342
SVC 0.09850107066381156
Dtree 0.09702353986085621
XGB 0.6048917780752783
KNN 0.11406341223896665


In [131]:
# Hence, I would like to submit my XGBoost model for predictions
# xgb.XGBRegressor(objective='reg:squarederror')

In [132]:
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, Y_train)
score = model.score(X_test, Y_test)
print(name, score)
print(model.feature_importances_)
# the Prior txn is most important predictor

KNN 0.6048917780752783
[0.01447284 0.02692628 0.0225207  0.45858502 0.05773668 0.03756683
 0.0589546  0.         0.02662461 0.07723869 0.02132316 0.03727756
 0.13423076 0.02654232]


In [133]:
from google.colab import drive
drive.mount('drive')

df2x = df2[df2['Date_txn'] == df2['Date_last']]
df2x['date'] = pd.to_datetime(df2x['Date_txn'])
df2x['date'] = df2x['date'].map(dt.datetime.toordinal) 
df2x['date'] = df2x['date'] - df2x['date'].min()
df3 = df2x.drop(columns=['Date_first','Date_last','prev_txn','prev_txn-1','prev_txn-2','prev_txn-3','prev_txn-4','prev_txn-5','prev_txn-6','prev_txn-7','prev_txn-8','prev_txn-9','prev_txn-10','Date_txn'])
df3['Member'] = df3['Member'].str[1:]
df3['Member'] = df3['Member'].apply(pd.to_numeric)
X = df3.drop(columns=["diff_bw_txns"])

predictions = model.predict(X)
arr = []
listz = df2[df2['Date_txn'] == df2['Date_last']]['Date_txn'].tolist()
for i in range(len(predictions)):
    arr.append(listz[i] + timedelta(days=int(predictions[i])))

df_final1 = pd.DataFrame(arr, columns = ['next_txn'], index = df2x['Member'])
df_final1.reset_index(level=0, inplace=True)

df_final1.to_csv('submission_part1.csv')
!cp submission_part1.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

# Part 2

In [134]:
df = pd.read_excel('bigbasketcasedataset.xlsx', 'Sheet1')
df.head()

,Member,Order,SKU,Date,Description
0,M09736,6468572,34993740,2014-09-22,Other Sauces
1,M09736,6468572,15669800,2014-09-22,Cashews
2,M09736,6468572,34989501,2014-09-22,Other Dals
3,M09736,6468572,7572303,2014-09-22,Namkeen
4,M09736,6468572,15669856,2014-09-22,Sugar


In [135]:
# For this question I have decided to use the CPT Algorithm

df_all = df.sort_values(['Member', 'Date','SKU'], ascending=[True, True, True])
df_all= df_all.groupby('Member')['SKU'].apply(list).reset_index(name='OrderSKUs')

In [136]:
# Preprocessing

df_train = df_all.copy()
df_test = df_all.copy()

df_train['OrderSKUs'] = df_train['OrderSKUs'].apply(lambda x: x[:-10])
df_test['OrderSKUs'] = df_test['OrderSKUs'].apply(lambda x: x[-10:])

In [137]:
df_train.tail(10)

,Member,OrderSKUs
96,M78720,"[7572303, 7617882, 15668468, 15668478, 1566977..."
97,M82651,"[7543241, 7572307, 7572312, 7587494, 7625765, ..."
98,M84827,"[15668381, 15668416, 15668451, 7587489, 763096..."
99,M86304,"[7580780, 15668381, 15668458, 15668475, 156683..."
100,M86572,"[7569801, 7569805, 7569806, 7570488, 7575487, ..."
101,M90375,"[7581918, 15668379, 15668457, 15668474, 156697..."
102,M91098,"[7631721, 15668451, 34988771, 92377117, 759872..."
103,M96365,"[15668685, 15669778, 15669803, 15669863, 75808..."
104,M99030,"[7753732, 15668383, 15668453, 15668455, 156684..."
105,M99206,"[7624790, 15668451, 15668462, 15668468, 156684..."


In [138]:
df_test.tail(10)

,Member,OrderSKUs
96,M78720,"[7628085, 7718555, 15668377, 15668458, 1566846..."
97,M82651,"[15669765, 15669784, 15669856, 15669861, 15669..."
98,M84827,"[21410625, 21410640, 21410666, 34938526, 15669..."
99,M86304,"[15668377, 15668380, 15668381, 15668460, 15668..."
100,M86572,"[15668377, 15668465, 15668469, 15668520, 15669..."
101,M90375,"[15669878, 15669885, 15669988, 15668375, 15668..."
102,M91098,"[34987220, 34991198, 7580802, 15668380, 156683..."
103,M96365,"[15668465, 93091858, 15669811, 15668375, 15668..."
104,M99030,"[15668460, 15668688, 15669776, 15669777, 15669..."
105,M99206,"[15668473, 15668478, 15668688, 15669766, 15669..."


In [139]:
df_all.head()

,Member,OrderSKUs
0,M04158,"[7590864, 15668520, 15668687, 15669869, 156702..."
1,M08075,"[15668449, 15668469, 15669863, 15668416, 15669..."
2,M09303,"[7572314, 7718555, 15668688, 15669957, 1566997..."
3,M09736,"[7569805, 7586147, 15668375, 15668379, 1566846..."
4,M12050,"[7569801, 7570758, 7586307, 15668416, 15668452..."


In [140]:
# Attempt 1

model = Cpt()

# I will train using DF_all for top 90 of the individuals
# Then I will predict using DF_train for the bottom 10 of the individuals
# And validate the results with DF_test

output = []
input = []
model.fit(df_all['OrderSKUs'].head(90))
for item1 in df_test['OrderSKUs'].tail(10):
    input.append(item1)
for item in df_train['OrderSKUs'].tail(10):
    output.append(item)

op = model.predict_k(output, 10)

In [141]:
# Source for code snippet: 
# https://www.geeksforgeeks.org/python-count-of-matching-elements-among-lists-including-duplicates/

res = 0
for i in range(len(op)):
    for ele in input[i]:
        if ele in op[i]:
            res += 1

print("Accuracy:", res/100)

# This does not predict very well given how we trained using different customers
# and tested on different customers.

Accuracy: 0.02


In [142]:
# Attempt 2
# Training and testing on same customers, while removing last 10 orders

model = Cpt()

output = []
input = []
model.fit(df_train['OrderSKUs'])
for item1 in df_test['OrderSKUs']:
    input.append(item1)
for item in df_train['OrderSKUs']:
    output.append(item)

op = model.predict_k(output, 10)

res = 0
for i in range(len(op)):
    for ele in input[i]:
        if ele in op[i]:
            res += 1

# Measuring the number of SKUs correctly predicted across all customers. 
print("Accuracy:", res/(10*len(op)))

# This predicts better
# The error can further be reduced by compute_noisy_items and removing

Accuracy: 0.13584905660377358


In [143]:
model = Cpt()

output = []
model.fit(df_all['OrderSKUs'])
for item in df_train['OrderSKUs']:
    output.append(item)

op = model.predict_k(output, 10)
df_final2 = pd.DataFrame(op, columns = ['SKU1','SKU2','SKU3','SKU4','SKU5','SKU6','SKU7','SKU8','SKU9','SKU10'], index = df_all['Member'])
df_final2.reset_index(level=0, inplace=True)

df_final2.to_csv('submission_part2.csv')
!cp submission_part2.csv "drive/My Drive/"

In [144]:
# another basic tactic using frequency distributions

df[['Member','SKU','Date']].groupby(['Member','SKU'], as_index = True).count().sort_values(by=['Member','Date'], ascending=[True,False]).head(10)

# df_train.groupby(['Member','SKU'])['SKU'].count()

# Out of these, 4 entries match the last 10 orders
# which suggests a 40% accuracy, but this isn't very ML friendly.

Date
Member SKU           
M04158 15668378    29
       15668688    21
       15668478    19
       15668520    19
       15668458    18
       15668449    16
       15668687    16
       15668381    14
       15669864    13
       7629390     12

In [145]:
# Other approaches I had planned:

# https://www.kdnuggets.com/2019/11/markov-chains-train-text-generation.html
# https://www.youtube.com/watch?v=WL6DZPsGPt8&ab_channel=SandeepBhutani
# https://www.geeksforgeeks.org/apriori-algorithm/